## 4.2.1 python作用域基础

### 4.2.1.1 作用域细节

### 4.2.1.2 变量名解析

### 4.2.1.3 作用域实例

### 4.2.1.4 内置作用域

## 4.2.2 global语句

### 4.2.2.1 最小化全局变量

### 4.2.2.2 最小化跨文件修改

### 4.2.2.3 其他访问全局变量的方式

## 4.2.3 作用域和嵌套参数

### 4.2.3.1 嵌套作用域的细节

### 4.2.3.2 嵌套作用域举例

### 4.2.3.3 工厂函数

### 4.2.3.4 使用默认值

## 4.2.4 nonlocal

### 4.2.4.1 nonlocal基础

### 4.2.4.2 nonlocal应用

### 4.2.4.3 变量状态

### 4.2.4.4 全局变量

### 4.2.4.5 类的状态

### 4.2.4.6 定制open